In [1]:
import pandas as pd

df = pd.read_json("../../datas/zap/detected.json", lines=True)
# df = pd.read_json("../detected.json", lines=True)

df.head(5)

,method,path,headers,uuid,peer,status,cookies,response_msg,timestamp,post_data
0,GET,/,"{'host': '127.0.0.1', 'user-agent': 'curl/7.81...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 60626}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:41:17.463548,NaN
1,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50540}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.438251,NaN
2,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50540}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.499190,NaN
3,GET,/robots.txt,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50542}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.496824,NaN
4,GET,/,"{'host': '127.0.0.1', 'user-agent': 'Mozilla/5...",abbc944d-6737-46da-a56a-7300397feb8f,"{'ip': '172.18.0.1', 'port': 50546}",200,{'sess_uuid': None},"{'version': '0.6.0', 'response': {'message': {...",2025-06-14 09:42:02.501211,NaN


In [2]:
attacks_matrix = []
tmp = []
sess_uuids_attacks = {
  # "{sess_uuid}": []
}
sess_uuids = []

for _, row in df.iterrows():
  sess_uuid = row.get("cookies", {}).get("sess_uuid") or row.get("response_msg", {}).get("response", {}).get("message", {}).get("sess_uuid")
  detection = row.get("response_msg", {}).get("response", {}).get("message", {}).get("detection", {})
  attack_type = detection.get("name", "")

  if sess_uuid in sess_uuids:
    sess_uuids_attacks[sess_uuid].append(attack_type)
  else:
    sess_uuids_attacks[sess_uuid] = [attack_type]
    sess_uuids.append(sess_uuid)

attacks_matrix = list(sess_uuids_attacks.values())
# for paths in sess_uuids_attacks.values():
#   attacks_matrix.append(paths)

sess_uuids_attacks
# attacks_matrix
# print(len(attacks_matrix), len(sess_uuids_paths.keys()))

{'c58bf9d6-7a8a-4b9c-8dcd-81406de2bbd6': ['index'],
 'cf902971-14ec-45f8-9839-6fb5e589eb71': ['index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'unknown',
  'unknown',
  'index',
  'unknown',
  'unknown',
  'unknown',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'unknown',
  'unknown',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'unknown',
  'unknown',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'index',
  'unknown',
  'unknown',
  'index',
  'index',
  'unknown',
  'unknown',
  

In [3]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth

te = TransactionEncoder()
te_ary = te.fit(attacks_matrix).transform(attacks_matrix)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head(10)

,cmd_exec,crlf,index,lfi,sqli,template_injection,unknown,xss
0,False,False,True,False,False,False,False,False
1,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True
3,True,True,True,True,True,False,True,True
4,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True
6,True,True,True,True,True,False,True,True
7,True,False,True,True,True,False,True,True
8,True,False,True,True,True,False,True,True
9,False,False,True,False,False,False,False,False


In [5]:
# frequent_itemsets = fpgrowth(df, min_support=0.3, use_colnames=True)
frequent_itemsets = fpgrowth(df, min_support=0.1, use_colnames=True)
frequent_itemsets["length"] = frequent_itemsets["itemsets"].apply(lambda x: len(x))
### alternatively:
#frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
#frequent_itemsets = fpmax(df, min_support=0.6, use_colnames=True)

frequent_itemsets

,support,itemsets,length
0,1.000000,(index),1
1,0.346154,(xss),1
2,0.346154,(unknown),1
3,0.307692,(sqli),1
4,0.307692,(cmd_exec),1
...,...,...,...
250,0.153846,"(cmd_exec, crlf, template_injection, unknown, ...",7
251,0.153846,"(crlf, template_injection, unknown, sqli, inde...",7
252,0.153846,"(cmd_exec, crlf, template_injection, unknown, ...",7
253,0.153846,"(cmd_exec, template_injection, unknown, sqli, ...",7


In [6]:
large_frequent_itemsets = frequent_itemsets[frequent_itemsets["length"] >= 3]
large_frequent_itemsets

,support,itemsets,length
11,0.307692,"(unknown, index, xss)",3
15,0.307692,"(unknown, sqli, xss)",3
16,0.307692,"(sqli, unknown, index)",3
17,0.307692,"(sqli, xss, index)",3
18,0.307692,"(sqli, unknown, index, xss)",4
...,...,...,...
250,0.153846,"(cmd_exec, crlf, template_injection, unknown, ...",7
251,0.153846,"(crlf, template_injection, unknown, sqli, inde...",7
252,0.153846,"(cmd_exec, crlf, template_injection, unknown, ...",7
253,0.153846,"(cmd_exec, template_injection, unknown, sqli, ...",7


In [11]:
import psycopg2

conn = psycopg2.connect(database="web_honeypot_zap", user = "postgres", password = "admin", host = "127.0.0.1", port = "5432")

print("Opened database successfully")

Opened database successfully


In [9]:
# create table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [10]:
# insert data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully


In [12]:
# create large data table
cur = conn.cursor()
cur.execute('''CREATE TABLE assoc_rules_attack_types_large (
            ID INT PRIMARY KEY     NOT NULL,
            SUPPORT           REAL    NOT NULL,
            ATTACK_TYPE            VARCHAR(255)     NOT NULL);''')

print("Table created successfully")

conn.commit()

Table created successfully


In [13]:
# insert large data

cur = conn.cursor()

insert_query = """
    INSERT INTO assoc_rules_attack_types_large (ID, SUPPORT, ATTACK_TYPE)
    VALUES (%s, %s, %s)
"""

for idx, row in large_frequent_itemsets.iterrows():
    cur.execute(
        insert_query,
        (int(idx), float(row["support"]), str(list(row["itemsets"])))
    )

conn.commit()
print("Records created successfully")
conn.close()

Records created successfully
